In [18]:
import pandas as pd
import numpy as np

    Assume a two-stage Free Cash Flow model. The model breaks down the valuation into two parts: the Growth Phase and the Terminal Value calculation. The Growth Phase is the initial period where the cash flows grows at a non-constant rate. The Terminal Value represents the value of cash flow continuing into perpetuity from the end of the growth phase, growing at a constant rate. 

In [25]:
data = {
    "Period": ["2024", "2025", "2026", "2027", "2028"],
    "Profit before tax": [123280, 145960, 170222, 195821, 211089],
    "Net interest income/expense": [-182, -198, -216, -234, -238],
    "Depreciation & Amortization": [10336, 11418, 11969, 12821, 13682],
    "EBITDA": [133798, 157577, 182406, 208876, 225009],
    "Tax": [18146, 21484, 25055, 28823, 31070],
    "Capital Expenditure": [-11620, -12718, -13839, -14968, -15267],
    "Change in Net Working Capital": [-4020, -905, 1619, 1232, 2239],
    "Free Cash Flow (FCF)": [100012, 124280, 141894, 163853, 176433]
}
df = pd.DataFrame(data)
df.set_index("Period", inplace=True)
data_fcf = df[['Free Cash Flow (FCF)']]
data_fcf

,Free Cash Flow (FCF)
Period,
2024,100012
2025,124280
2026,141894
2027,163853
2028,176433


    Then, to calculate the Weighted Average Cost of Capital (WACC), we assume the market risk-free rate to as 4.1%, the market excess return as 9.2%, and beta to be 0.9.
    Series of parameters, including cost of equity and cost of debt are given as following.

In [36]:
# We then define the growth_exit to be five years.
growth_exit = 5
risk_free_rate = 0.041
expected_market_return = 0.092
beta = 0.9
cost_of_equity = risk_free_rate + beta * (expected_market_return - risk_free_rate)

cost_of_debt = 0.043
tax_rate = 0.15
after_tax_cost_of_debt = cost_of_debt * (1 - tax_rate)

debt_ratio = 0.029
equity_ratio = 1 - debt_ratio

wacc = (cost_of_equity * equity_ratio) + (after_tax_cost_of_debt * debt_ratio)
print(f'The cost of equity is {cost_of_equity}, the after tax cost of debt is {after_tax_cost_of_debt}，WACC为{wacc}。')

The cost of equity is 0.0869, the after tax cost of debt is 0.03655，WACC为0.08543985000000001。


    Then, we discount the cash flows within the growth phase.

In [37]:
discount_factor = []
for i in range(1, growth_exit + 1):
    discount = [1 / ((1 + wacc) ** i)]
    discount_factor.append(discount)

In [38]:
discount_factors_array = [factor[0] for factor in discount_factor]
df['Present Value of Free Cash Flow'] = (df['Free Cash Flow (FCF)'] * discount_factors_array).round()
df_discounted = df[['Free Cash Flow (FCF)','Present Value of Free Cash Flow']]
df_discounted

,Free Cash Flow (FCF),Present Value of Free Cash Flow
Period,,
2024,100012,92140.0
2025,124280,105485.0
2026,141894,110955.0
2027,163853,118041.0
2028,176433,117098.0


    Then, we deal with the calculation for Terminal Value. The Terminal Value is calculated at the end of the projected period (end of Year 2028) using the last projected discounted free cash flow 117098. Assume the constant perpetuity growth rate is 0.04.

In [43]:
perpetuity_rate = 0.04
end_of_2028 = df_discounted.iloc[-1, 1]
terminal_value = (end_of_2028 * (1 + perpetuity_rate)) / (wacc - perpetuity_rate) 
terminal_value

4387812.019625944

    Finally, summing up the cash flows and the terminal value to get an estimate of the company’s value today, based on future cash flow projections and the assumptions used.

In [50]:
total_pv = sum(df_discounted['Present Value of Free Cash Flow']) + terminal_value
total_pv = round(total_pv, 2)
print(f'The total present value is {total_pv}')

The total present value is 4931531.02
